In [ ]:
!pip install -q kokoro>=0.9.4 soundfile
!pip install uvicorn
!pip install pyngrok
!pip install fastapi
# !apt-get -qq -y install espeak-ng > /dev/null 2>&1
from kokoro import KPipeline
from IPython.display import display, Audio
import soundfile as sf
from pyngrok import ngrok, conf

In [2]:
def speech_to_text(text, language):
  pipeline = KPipeline(lang_code=language)
  generator = pipeline(text, voice='af_heart')
  for i, (gs, ps, audio) in enumerate(generator):
    print(i, gs, ps)
    display(Audio(data=audio, rate=24000, autoplay=i==0))
    sf.write(f'{i}.wav', audio, 24000)

In [ ]:
conf.get_default().auth_token = ""

In [4]:
import datetime

In [ ]:
import os
import tempfile
import uvicorn
from fastapi import FastAPI, Request, HTTPException
from fastapi.responses import FileResponse, JSONResponse
from pyngrok import ngrok
import soundfile as sf
from pydantic import BaseModel
import nest_asyncio
import datetime

app = FastAPI()
port = 5000

ngrok.kill()

# Ngrok туннель
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")


class AudioRequest(BaseModel):
    text: str
    language: str = "en"

@app.get("/")
async def logg():
    print('get 200')
    return 200

@app.post("/audio")
async def generate_audio(request_data: AudioRequest):
    try:
        text = request_data.text
        language = request_data.language
        print(text, language, datetime.datetime.now())
        # Временный WAV-файл
        temp_file = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)

        print('Start generation', datetime.datetime.now())
        # Генерация аудио
        pipeline = KPipeline(lang_code=language)
        generator = pipeline(text, voice='af_heart')
        print('End generation', datetime.datetime.now())

        # Берём первый сегмент
        for _, _, audio in generator:
            sf.write(temp_file.name, audio, 24000)
            break

        return FileResponse(
            path=temp_file.name,
            media_type='audio/wav',
            filename='output.wav'
        )

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


@app.get("/audio")
async def get_usage():
    return JSONResponse(content={"message": "Use POST /audio with JSON {text, language}"})


nest_asyncio.apply()
uvicorn.run(app, port=port)